# Computing daily ONN index on Binance

## Getting all available symbols on Binance

In this section, we just get all available assets on Binance, generating at the end a file named `all-binance-tokens.csv`. Each line in this file contains the asset symbol two times, separated by comma (e.g. `ETH,ETH`). This is because we will use it afterwards in the __[`bnbFetchCoinData.py`](https://github.com/srmq/bnbfetch-coin-data)__ script, which expects the CSV file of lines in the format `asset_description,asset_symbol`, and we don't care about descriptions here.

In [18]:
import httpx
import pandas as pd

In [2]:
binanceBaseURL = 'https://api.binance.com'

In [3]:
async def getExchangeInfo():
    spotExchangeInfoURL = binanceBaseURL + "/api/v3/exchangeInfo"

    async with httpx.AsyncClient() as client:
        r = await client.get(spotExchangeInfoURL)
    return r.json()

In [4]:
exchangeInfo = await getExchangeInfo()

In [5]:
#blacklist = None
#with open('blacklist.txt', 'r') as f:
#    blacklist = f.read().split(',')
# blacklist should be disabled for now, we need coin data for every coin

In [6]:
baseAssets = set()
#for coinSymbol in exchangeInfo['symbols']:
#    if not (coinSymbol['baseAsset'].lower() in blacklist):
#        baseAssets.add(coinSymbol['baseAsset'])

In [7]:
for coinSymbol in exchangeInfo['symbols']:
    baseAssets.add(coinSymbol['baseAsset'])

In [8]:
def notLeveraged(asset):
    return not (asset.endswith('UP') or asset.endswith('DOWN') or asset.endswith('BEAR') or asset.endswith('BULL'))

In [9]:
baseAssets = set(filter(notLeveraged, baseAssets))

In [10]:
baseAssets

{'1INCH',
 'AAVE',
 'ACA',
 'ACH',
 'ACM',
 'ADA',
 'ADX',
 'AE',
 'AERGO',
 'AGI',
 'AGIX',
 'AGLD',
 'AION',
 'AKRO',
 'ALCX',
 'ALGO',
 'ALICE',
 'ALPACA',
 'ALPHA',
 'ALPINE',
 'AMB',
 'AMP',
 'ANC',
 'ANKR',
 'ANT',
 'ANY',
 'APE',
 'API3',
 'APPC',
 'AR',
 'ARDR',
 'ARK',
 'ARN',
 'ARPA',
 'ASR',
 'AST',
 'ASTR',
 'ATA',
 'ATM',
 'ATOM',
 'AUCTION',
 'AUD',
 'AUDIO',
 'AUTO',
 'AVA',
 'AVAX',
 'AXS',
 'BADGER',
 'BAKE',
 'BAL',
 'BAND',
 'BAR',
 'BAT',
 'BCC',
 'BCD',
 'BCH',
 'BCHA',
 'BCHABC',
 'BCHSV',
 'BCN',
 'BCPT',
 'BDOT',
 'BEAM',
 'BEL',
 'BETA',
 'BETH',
 'BGBP',
 'BICO',
 'BIFI',
 'BKRW',
 'BLZ',
 'BNB',
 'BNT',
 'BNX',
 'BOND',
 'BOT',
 'BQX',
 'BRD',
 'BSW',
 'BTC',
 'BTCB',
 'BTCST',
 'BTG',
 'BTS',
 'BTT',
 'BTTC',
 'BURGER',
 'BUSD',
 'BZRX',
 'C98',
 'CAKE',
 'CDT',
 'CELO',
 'CELR',
 'CFX',
 'CHAT',
 'CHESS',
 'CHR',
 'CHZ',
 'CITY',
 'CKB',
 'CLOAK',
 'CLV',
 'CMT',
 'CND',
 'COCOS',
 'COMP',
 'COS',
 'COTI',
 'COVER',
 'CREAM',
 'CRV',
 'CTK',
 'CTSI',
 'CTXC

In [11]:
with open('all-binance-tokens.csv', 'w') as f:
    for asset in baseAssets:
        f.write(f"{asset},{asset}\n")

Now that we have our `all-binance-tokens.csv` file, we can download, for instance, the complete historical data for all assets on Binance, using something like:

`python <path_to_script>/bnbFetchCoinData.py --coins all-binance-tokens.csv --interval 1d`

This will create a `coindata` directory under the current one with everything we need.

## Importing Data to QuestDB

We will use __[QuestDB](https://questdb.io/)__ in order to facilitate data manipulation and keep memory requirements low (we won't need to load everything on RAM). First we will create a table for holding the data. Here is what we did:

`CREATE TABLE binance_daily(symb SYMBOL capacity 2048, open_time TIMESTAMP, high DOUBLE, low DOUBLE, close DOUBLE, volume DOUBLE, close_time TIMESTAMP, quote_asset_vol DOUBLE, num_trades LONG, takerbuy_baseasset_vol DOUBLE, takerbuy_quoteasset_vol DOUBLE), index(symb) timestamp(open_time) PARTITION BY YEAR;`

Now we will import the data (that went to the `coindata` subdirectory)...

In [1]:
import glob

allCSVFiles = glob.glob('./coindata/*.csv')

In [2]:
import httpx
import csv

host = 'http://localhost:9000'

async def questInsert(symbol: str, filename: str, lotSize=300):
    async def executeQuery(valStrings):
        sql_query = "INSERT INTO binance_daily VALUES"
        sql_query += valStrings[0]
        for i in range(1, len(valStrings)):
            sql_query += f",{valStrings[i]}"
        query_params = {'query': sql_query, 'fmt': 'json'}
        async with httpx.AsyncClient() as client:
            await client.get(host + '/exec', params=query_params)
        
    with open(filename, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        valStrings = []
        for row in reader:
            if len(valStrings) == lotSize:
                await executeQuery(valStrings)
                valStrings = []
            val = f"(\'{symbol}\', {int(row['Open time'])*1000}, {float(row['High'])}, {float(row['Low'])}, {float(row['Close'])}, {float(row['Volume'])}, {int(row['Close time'] + '999')}, {float(row['Quote asset volume'])}, {int(row['Number of trades'])}, {float(row['Taker buy base asset volume'])}, {float(row['Taker buy quote asset volume'])})"
            valStrings.append(val)
        if len(valStrings) > 0:
            await executeQuery(valStrings)

In [3]:
from pathlib import Path
i = 0
for csvFile in allCSVFiles:
    symb = Path(csvFile).name.split('.')[0]
    await questInsert(symb, csvFile)
    i += 1
    if i % 100 == 0:
        print(f"Inserted {i} symbols out of {len(allCSVFiles)}")    

Inserted 100 symbols out of 1819
Inserted 200 symbols out of 1819
Inserted 300 symbols out of 1819
Inserted 400 symbols out of 1819
Inserted 500 symbols out of 1819
Inserted 600 symbols out of 1819
Inserted 700 symbols out of 1819
Inserted 800 symbols out of 1819
Inserted 900 symbols out of 1819
Inserted 1000 symbols out of 1819
Inserted 1100 symbols out of 1819
Inserted 1200 symbols out of 1819
Inserted 1300 symbols out of 1819
Inserted 1400 symbols out of 1819
Inserted 1500 symbols out of 1819
Inserted 1600 symbols out of 1819
Inserted 1700 symbols out of 1819
Inserted 1800 symbols out of 1819


Now QuestDB has ingested all data. Let's play!

## Playing with the data

In [1]:
import pandas as pd
import httpx

In [2]:
host = 'http://localhost:9000'

In [3]:
async def daysTradedInMonthBySymbol(year: int, month: int) -> pd.DataFrame:
    assert(month >= 1 and month <= 12)
    assert(year >= 2000 and year <= 9999)
    if month == 12:
        nextMonth = 1
        nextMonthYear = year + 1
    else:
        nextMonth = month + 1
        nextMonthYear = year
    sql_query = "select symb, count(*) as daystraded from 'binance_daily' where open_time >= '%d-%02d-01T00:00:00.000000Z' and open_time < '%d-%02d-01T00:00:00.000000Z' and num_trades >= 1 group by symb"%(year, month, nextMonthYear, nextMonth)
    query_params = {'query': sql_query, 'fmt': 'json'}
    async with httpx.AsyncClient() as client:
            r = await client.get(host + '/exec', params=query_params)
    jsonR = r.json()
    return pd.DataFrame(columns=['symb', 'daysTraded'], data=jsonR['dataset'])

In [4]:
import json
with open('coindata/coin-metadata.json', 'r') as f:
    coinMetadata = json.loads(f.read())
symbols = coinMetadata['exchange-info']['symbols']
symbolDict = {}
symbolsByBaseDict = {}
for symbol in symbols:
    symbolDict[symbol['symbol']] = symbol
    baseSymbols = symbolsByBaseDict.get(symbol['baseAsset'], [])
    baseSymbols.append(symbol)
    symbolsByBaseDict[symbol['baseAsset']] = baseSymbols
del symbols

In [5]:
from typing import Set
# It starts as all assets with TRADING status
async def initialAssetCandidateSet() -> Set[str]:
    candidateAssets = set()
    for symbol in symbolDict:
        baseAsset = symbolDict[symbol]['baseAsset']
        if symbolDict[symbol]['status'] == 'TRADING':
            if not baseAsset.endswith('UP') and not baseAsset.endswith('DOWN') and not baseAsset.endswith('BEAR') and not baseAsset.endswith('BULL'):
                candidateAssets.add(baseAsset)
    
    return candidateAssets

In [6]:
def getTradeSymbols(baseAssetSet):
    symbols = []
    for base in baseAssetSet:
        baseInfo = symbolsByBaseDict[base]
        for el in baseInfo:
            symbols.append(el['symbol'])
    return symbols

In [7]:
async def getMedianUSDTTradeVol(baseAssetSet: Set[str], isoInstantFrom: str, isoInstantTo: str) -> pd.DataFrame:
    def listToStr(l):
        result = l[0]
        for i in range(1, len(l)):
            result += f", {l[i]}"
        return result
    
    symbolList = getTradeSymbols(baseAssetSet)
    columns = ['symb', 'open_time', 'close', 'volume', 'close_time', 'quote_asset_vol']
    # sql_query = "SELECT %s FROM 'binance_daily' WHERE open_time >= '%s' and open_time < '%s' AND symb IN (%s)"%(listToStr(columns), isoInstantFrom, isoInstantTo, str(symbolList)[1:len(str(symbolList))-1])
    sql_query = "SELECT %s FROM 'binance_daily' WHERE open_time >= '%s' and open_time < '%s'"%(listToStr(columns), isoInstantFrom, isoInstantTo)    
    #print(sql_query)
    query_params = {'query': sql_query, 'fmt': 'json'}
    async with httpx.AsyncClient() as client:
            r = await client.get(host + '/exec', params=query_params)
    jsonR = r.json()
    df = pd.DataFrame(columns=columns, data=jsonR['dataset'])
    del r
    del jsonR
    df['baseAsset'] = df.apply(lambda row: symbolDict[row['symb']]['baseAsset'], axis = 1)
    df['quoteAsset'] = df.apply(lambda row: symbolDict[row['symb']]['quoteAsset'], axis = 1)
    
    def genCoinGraph():
        from collections import defaultdict
        coinGraph = defaultdict(set)
        for key in symbolDict:
            symbol = symbolDict[key]
            coinGraph[symbol['baseAsset']].add(symbol['quoteAsset'])
            coinGraph[symbol['quoteAsset']].add(symbol['baseAsset'])
        return coinGraph
    
    coinGraph = genCoinGraph()
    
    # We will use USDT to compare apples to apples
    def shortestPathToUSDT(fromBaseAsset):
        
        if fromBaseAsset == 'USDT':
            return [fromBaseAsset]
        parent = {}
        explored = set()
        frontier = [fromBaseAsset]
        def solution(baseAsset):
            if parent.get(baseAsset) is None:
                return [baseAsset]
            else:
                result = solution(parent[baseAsset])
                result.append(baseAsset)
                return result

        while len(frontier) > 0:
            node = frontier.pop(0)
            explored.add(node)
            for coin in coinGraph[node]:
                if coin not in explored and coin not in frontier:
                    parent[coin] = node
                    if coin == 'USDT':
                        return solution(coin)
                    else:
                        frontier.append(coin)
        assert False, "Should never get here"    
        
    valInUSDT = []
    for (_, row) in df.iterrows():
        # print("(Base asset: %s, Quote asset: %s)"%(row['baseAsset'],row['quoteAsset']))
        if row['quoteAsset'] == 'USDT':
            val = row['volume'] * row['close']
        else:
            path = shortestPathToUSDT(row['quoteAsset'])
            multiplier = 1.0
            assert len(path) > 1
            i = 0
            while i + 1 < len(path):
                dfFiltered = df[(df['baseAsset'] == path[i])
                                     & (df['quoteAsset'] == path[i + 1])
                                     & (df['open_time'] == row['open_time'])]
                if dfFiltered.shape[0] > 0:
                    # print(dfFiltered)
                    pathRow = dfFiltered.iloc[0]
                    multiplier *= pathRow['close']
                else:
                    dfFiltered = df[(df['baseAsset'] == path[i + 1]) 
                            & (df['quoteAsset'] == path[i])
                            & (df['open_time'] == row['open_time'])]
                    # print(dfFiltered)
                    pathRow = dfFiltered.iloc[0]
                    multiplier *= 1.0 / pathRow['close']
                i += 1
            assert path[i] == 'USDT'
            val = row['volume'] * row['close'] * multiplier
        valInUSDT.append(val)
    df.insert(df.shape[1], 'valInUSDT', valInUSDT)
    del valInUSDT

    df = df[['open_time', 'baseAsset', 'valInUSDT']].groupby(['baseAsset', 'open_time'], as_index=False).sum()    
    df = df[['baseAsset', 'valInUSDT']].groupby(['baseAsset'], as_index=False).median().sort_values(['valInUSDT'], ascending=[False], ignore_index=True)
    return df

In [8]:
def getBlacklistSet() -> Set[str]:
    blacklist = None
    with open('blacklist.txt', 'r') as f:
        blacklist = f.read().split(',')
    return set(blacklist)

In [9]:
async def daysTradedBetweenBySymbol(isoInstantFrom: str, isoInstantTo: str) -> pd.DataFrame:
    sql_query = "select symb, count(*) as daystraded from 'binance_daily' where open_time >= '%s' and open_time < '%s' and num_trades >= 1 group by symb"%(isoInstantFrom, isoInstantTo)
    query_params = {'query': sql_query, 'fmt': 'json'}
    async with httpx.AsyncClient() as client:
            r = await client.get(host + '/exec', params=query_params)
    jsonR = r.json()
    return pd.DataFrame(columns=['symb', 'daysTraded'], data=jsonR['dataset'])

In [10]:
from datetime import date
from typing import List

# we will consider that an eligible asset should have at least one trade
# pair that was negotiated every day of the month
async def symbolsTradedEveryDay(isoInstantFrom: str, isoInstantTo: str) -> List[str]:
    fromDate = date.fromisoformat(isoInstantFrom[:10])
    toDate = date.fromisoformat(isoInstantTo[:10])
    daysBetween = abs(toDate - fromDate).days    
    df = await daysTradedBetweenBySymbol(isoInstantFrom, isoInstantTo)
    return df[df['daysTraded']==daysBetween]['symb'].tolist()

In [11]:
async def assetsTradedEveryDay(isoInstantFrom: str, isoInstantTo: str) -> Set[str]:
    result = set()
    symbols = await symbolsTradedEveryDay(isoInstantFrom, isoInstantTo)
    for symbol in symbols:
        symbolInfo = symbolDict[symbol]
        result.add(symbolInfo['baseAsset'])
        result.add(symbolInfo['quoteAsset'])
    return result

In [12]:
async def eligibleAssetsAndMedianVolumes(isoInstantFrom: str, isoInstantTo: str) -> pd.DataFrame:
    initialAssets = await initialAssetCandidateSet()
    df = await getMedianUSDTTradeVol(initialAssets, isoInstantFrom, isoInstantTo)
    blackListSet = getBlacklistSet()
    df = df[~df['baseAsset'].str.lower().isin(blackListSet)]
    everydayAssets = await assetsTradedEveryDay(isoInstantFrom, isoInstantTo)
    df = df[df['baseAsset'].isin(everydayAssets)]
    maxVolume = df['valInUSDT'][0]
    df = df[df['valInUSDT'] >= (0.005 * maxVolume)]
    df.reset_index(inplace=True, drop=True)
    return df

In [13]:
eligibleAssets = await eligibleAssetsAndMedianVolumes('2020-11-01T00:00:00.000000Z', '2020-12-01T00:00:00.000000Z')
eligibleAssets

,baseAsset,valInUSDT
0,BTC,1.799611e+09
1,ETH,8.656754e+08
2,LINK,1.757122e+08
3,YFI,1.491913e+08
4,LTC,1.489384e+08
5,XRP,1.256029e+08
6,BNB,1.004437e+08
7,BCH,7.264516e+07
8,UNI,6.673704e+07
9,ADA,5.294544e+07


Here we have our eligibleAssets, now we will proceed to select among then the ones that will be part of the index constituents... For this we will need market cap data, which is not available from Binance API, so we will get them from Coin Gecko.

In [14]:
geckoBaseURL = "https://api.coingecko.com:443/api/v3"

In [15]:
def getBlacklistGeckoIds() :
    blackListGeckoIds = [
        'alphacoin', 
        'polyalpha-finance', 
        'aavegotchi-alpha', 
        'coinviewcap',
        'covicoin',
        'tron-bsc',
        'unicorn-token',
        'playground-waves-floor-index'
    ]
    return blackListGeckoIds

In [16]:
async def getGeckoCoinsInEligible(eligibleAssets: pd.DataFrame) -> pd.DataFrame:
    query_params = {'include_platform': 'false'}
    async with httpx.AsyncClient() as client:
        r = await client.get(geckoBaseURL + '/coins/list', params=query_params)
    geckoCoins = pd.DataFrame(r.json())
    geckoCoins = geckoCoins[~geckoCoins['id'].str.contains('-peg-', regex=False)]
    geckoCoins = geckoCoins[~geckoCoins['id'].str.contains('-wormhole', regex=False)]
    geckoCoins = geckoCoins[~geckoCoins['id'].isin(getBlacklistGeckoIds())]
    geckoCoins = geckoCoins[geckoCoins['symbol'].str.upper().isin(eligibleAssets['baseAsset'])]
    geckoCoins.sort_values(['symbol'], ascending=[True], ignore_index=True, inplace=True)
    return geckoCoins

In [17]:
def repeatedSymbolCoins(geckoCoins: pd.DataFrame) -> pd.DataFrame:
    geckoGrouped = geckoCoins.groupby(['symbol']).size().reset_index(name='counts')
    geckoRepeated = geckoGrouped[geckoGrouped['counts'] > 1]
    return geckoCoins[geckoCoins['symbol'].isin(geckoRepeated['symbol'])]

In [18]:
import dateutil
from dateutil.parser import parse
import datetime as dt
async def computeMedianDailyMarketCap(geckoCoinId: str, isoInstantFrom: str, isoInstantTo: str) -> float:
    dtFrom = parse(isoInstantFrom)
    dtTo = parse(isoInstantTo)
    epochSecondsFrom = dtFrom.timestamp()
    epochSecondsTo = dtTo.timestamp()
    query_params = {'vs_currency': 'usd', 'from': epochSecondsFrom, 'to': epochSecondsTo}
    async with httpx.AsyncClient() as client:
        r = await client.get(geckoBaseURL + f"/coins/{geckoCoinId}/market_chart/range", params=query_params)
    jsonR = r.json()
    df = pd.DataFrame(columns=['epoch', 'market_cap'], data=jsonR['market_caps'])
    df['isodate'] = df.apply(lambda row: dt.datetime.utcfromtimestamp(int(row['epoch'])/1000).isoformat()[:10] if (len(str(row['epoch'])) >= 13) else dt.datetime.utcfromtimestamp(int(row['epoch'])).isoformat()[:10], axis = 1)
    df.sort_values(['epoch'], ascending=[True], ignore_index=True, inplace=True)
    # last market cap is available in last row of above df FIXME TODO
    df = df[['isodate', 'market_cap']].groupby(['isodate'], as_index=False).last()
    return df['market_cap'].median()

In [19]:
import asyncio
async def selectIndexConstituents(isoInstantFrom: str, isoInstantTo: str, eligibleAssets: pd.DataFrame) -> pd.DataFrame:
    geckoCoins = await getGeckoCoinsInEligible(eligibleAssets)
    repeatedSymbols = repeatedSymbolCoins(geckoCoins)
    if (repeatedSymbols.shape[0] != 0):
        with pd.option_context('display.max_rows', None):
            print(repeatedSymbols)
        raise Exception("Symbols with more than one gecko id were found. You should revise getBlacklistGeckoIds() to add the undesired ones, duplicated symbols were printed above")
    else:
        indexConstituents = eligibleAssets.copy()
        eligibleAssets = None
        indexConstituents['geckoId'] = indexConstituents.apply(lambda row: geckoCoins[geckoCoins['symbol'] == row['baseAsset'].lower()]['id'].head(1).item(), axis = 1)
        medians = []
        for geckoId in indexConstituents['geckoId']:
            print(geckoId)
            await asyncio.sleep(4)
            medians.append(await computeMedianDailyMarketCap(geckoId, isoInstantFrom, isoInstantTo))
        indexConstituents['medianMarketcap'] = medians
        del medians
        totalMarketCap = indexConstituents['medianMarketcap'].sum()
        indexConstituents.sort_values(['medianMarketcap'], ascending=[False], ignore_index=True, inplace=True)
        indexConstituents['fracTotalMarketCap'] = indexConstituents.apply(lambda row: row['medianMarketcap']/totalMarketCap, axis = 1)
        accumulatedPerc = []
        cumul = 0.0
        for _, row in indexConstituents.iterrows():
          cumul += row['fracTotalMarketCap']
          accumulatedPerc.append(cumul)
        indexConstituents.insert(indexConstituents.shape[1], 'accumulatedPerc', accumulatedPerc)
        del accumulatedPerc
        lastIndex = 0
        for lastIndex, row in indexConstituents.iterrows():
            if row['fracTotalMarketCap'] < 0.005 and row['accumulatedPerc'] > 0.75:
                break
        indexConstituents = indexConstituents[:lastIndex]
        return indexConstituents        

In [20]:
await selectIndexConstituents('2020-11-01T00:00:00.000000Z', '2020-12-01T00:00:00.000000Z', eligibleAssets)

bitcoin
ethereum
chainlink
yearn-finance
litecoin
ripple
binancecoin
bitcoin-cash
uniswap
cardano
eos
polkadot
yfii-finance
aave
tron
swipe
sushi
vechain
curve-dao-token
monero
zcash
filecoin
tezos
ethereum-classic
neo
cosmos
stellar
reserve-rights-token
waves
dash
omisego
band-protocol
near
alpha-finance
havven
republic-protocol
civic
tellor


,baseAsset,valInUSDT,geckoId,medianMarketcap,fracTotalMarketCap,accumulatedPerc
0,BTC,1.799611e+09,bitcoin,3.061478e+11,0.731885,0.731885
1,ETH,8.656754e+08,ethereum,5.305535e+10,0.126835,0.858720
2,XRP,1.256029e+08,ripple,1.258803e+10,0.030093,0.888813
3,LINK,1.757122e+08,chainlink,5.041331e+09,0.012052,0.900865
4,BCH,7.264516e+07,bitcoin-cash,4.805316e+09,0.011488,0.912353
5,LTC,1.489384e+08,litecoin,4.428630e+09,0.010587,0.922940
6,DOT,3.833155e+07,polkadot,4.275687e+09,0.010222,0.933162
7,BNB,1.004437e+08,binancecoin,4.163545e+09,0.009953,0.943115
8,ADA,5.294544e+07,cardano,3.307420e+09,0.007907,0.951022
9,EOS,4.550082e+07,eos,2.455377e+09,0.005870,0.956892


In [ ]:
#computeLastCirculatingSupply(geckoCoinId: str, isoInstantFrom: str, isoInstantTo: str) -> float:
####lastCirculatingSupply is last marketcap divided by lastPrice